<a href="https://colab.research.google.com/github/pierre-brendan/PelotonIQ/blob/master/cyclist_profiles_code/Cyclist_profile_data_w_age.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# import packages
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

# start with non-looped method to get profiles
url = 'https://www.procyclingstats.com/rider/dylan-groenewegen'
html = requests.get(url).text
soup = BeautifulSoup(html, 'html5lib')

# GC method
spans = soup.find_all('li', {"class": "gc"})
gc = [span.get_text() for span in spans]
gc = ''.join(gc)
gc = re.sub('GC', '', gc)

# Classic
spans = soup.find_all('li', {"class": "classic"})
classic = [span.get_text() for span in spans]
classic = ''.join(classic)
classic = re.sub('One day races', '', classic)

# Time trial
spans = soup.find_all('li', {"class": "tt"})
tt = [span.get_text() for span in spans]
tt = ''.join(tt)
tt = re.sub('Time trial', '', tt)

# Sprint
spans = soup.find_all('li', {"class": "sprint"})
sprint = [span.get_text() for span in spans]
sprint = ''.join(sprint)
sprint = re.sub('Sprint', '', sprint)

# Climber
spans = soup.find_all('li', {"class": "climber"})
climber = [span.get_text() for span in spans]
climber = ''.join(climber)
climber = re.sub('Climber', '', climber)

# Rider
spans = soup.find_all('h1')
lines = [span.get_text() for span in spans]
rider2 = lines
rider2 = ''.join(rider2)
rider2 = re.sub("».*$", "", rider2)
rider2 = re.sub("  ", "", rider2)

# Team
team2 = lines
team2 = ''.join(team2)
team2 = re.sub('$.»*', '', team2)
try: 
  team2 =  team2.split('»')[1]
except Exception:
  team2 = ""
print(team2) 

# Age
spans = soup.find_all('div', {"class": "rdr-info-cont"})
lines = [span.get_text() for span in spans]
age = ''.join(lines)
try: 
  age = age[age.find("(")+1:age.find(")")]
except Exception:
  age = ""
print(age) 

In [0]:
# Age
spans = soup.find_all('div', {"class": "rdr-info-cont"})
lines = [span.get_text() for span in spans]
age = ''.join(lines)
try: 
  age = age[age.find("(")+1:age.find(")")]
except Exception:
  age = ""
print(age) 

Looped version

In [16]:
# Mount drive to get cyclist links
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# import packages
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from datetime import date

# Load in cyclist links for loop
df1 = pd.read_csv('/content/drive/My Drive/cycling_data/cyclist_links.csv')
urls = df1['proper_link']

#urls = ['https://www.procyclingstats.com/rider/alejandro-valverde', 'https://www.procyclingstats.com/rider/julian-alaphilippe']
appended_data = []

for i in urls:
  print(i)
  # start with non-looped method to get profiles
  url = i
  html = requests.get(url).text
  soup = BeautifulSoup(html, 'html5lib')

  # GC method
  spans = soup.find_all('li', {"class": "gc"})
  gc = [span.get_text() for span in spans]
  gc = ''.join(gc)
  gc = re.sub('GC', '', gc)

  # Classic
  spans = soup.find_all('li', {"class": "classic"})
  classic = [span.get_text() for span in spans]
  classic = ''.join(classic)
  classic = re.sub('One day races', '', classic)

  # Time trial
  spans = soup.find_all('li', {"class": "tt"})
  tt = [span.get_text() for span in spans]
  tt = ''.join(tt)
  tt = re.sub('Time trial', '', tt)

  # Sprint
  spans = soup.find_all('li', {"class": "sprint"})
  sprint = [span.get_text() for span in spans]
  sprint = ''.join(sprint)
  sprint = re.sub('Sprint', '', sprint)

  # Climber
  spans = soup.find_all('li', {"class": "climber"})
  climber = [span.get_text() for span in spans]
  climber = ''.join(climber)
  climber = re.sub('Climber', '', climber)

  # Rider
  spans = soup.find_all('h1')
  lines = [span.get_text() for span in spans]
  rider2 = lines
  rider2 = ''.join(rider2)
  rider2 = re.sub("».*$", "", rider2)
  rider2 = re.sub("  ", "", rider2)

  # Team
  team2 = lines
  team2 = ''.join(team2)
  team2 = re.sub('$.»*', '', team2)
  try: 
    team2 =  team2.split('»')[1]
  except Exception:
     team2 = ""

  # Age    
  spans = soup.find_all('div', {"class": "rdr-info-cont"})
  lines = [span.get_text() for span in spans]
  age = ''.join(lines)
  try: 
    age = age[age.find("(")+1:age.find(")")]
  except Exception:
    age = ""

  data = pd.DataFrame(
    {'Rider': [rider2],
     'Team': [team2],
     'GC': [gc],
     'TT': [tt],
     'Sprint': [sprint],
     'Climber': [climber],
     'Classic': [classic],
     'Age': [age]
    })

  # store DataFrame in list
  appended_data.append(data)

# see pd.concat documentation for more info
appended_data = pd.concat(appended_data)

# add crawl date
appended_data['crawl_date'] = date.today()

# Save the data file
appended_data.to_csv('rider_data.csv', index = False)


https://www.procyclingstats.com/rider/primoz-roglic
https://www.procyclingstats.com/rider/julian-alaphilippe
https://www.procyclingstats.com/rider/alejandro-valverde
https://www.procyclingstats.com/rider/jakob-fuglsang
https://www.procyclingstats.com/rider/egan-bernal
https://www.procyclingstats.com/rider/greg-van-avermaet
https://www.procyclingstats.com/rider/pascal-ackermann
https://www.procyclingstats.com/rider/elia-viviani
https://www.procyclingstats.com/rider/caleb-ewan
https://www.procyclingstats.com/rider/alexander-kristoff
https://www.procyclingstats.com/rider/matteo-trentin
https://www.procyclingstats.com/rider/peter-sagan
https://www.procyclingstats.com/rider/tadej-pogacar
https://www.procyclingstats.com/rider/bauke-mollema
https://www.procyclingstats.com/rider/mathieu-van-der-poel
https://www.procyclingstats.com/rider/oliver-naesen
https://www.procyclingstats.com/rider/emanuel-buchmann
https://www.procyclingstats.com/rider/sam-bennett
https://www.procyclingstats.com/rider/dy